In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#drop(5), ref
DROPREF = True


datafile = "../data/raw_data/KUICU220628_mod.xlsx"
MV=pd.read_excel(datafile, sheet_name = 'MV')
NMV=pd.read_excel(datafile, sheet_name = 'NMV')
print(MV.columns)
print(NMV.columns)

KeyboardInterrupt: 

In [ ]:
raw_data_MV = MV[['ID', 'PID','gender','death','age','mv', 'MV_time', 'R', 'HR_DATE', 'HR',
       'RR_DATE', 'RR', 'SBP_DATE', 'SBP', 'DBP_DATE', 'DBP', 'SpO₂_DATE',
       'SpO₂', 'BT_DATE', 'BT', 'CBC_DATE', 'Hb', 'Hct', 'RBC Count', 'MCV',
       'MCH', 'MCHC', 'WBC_Count', 'PLT_Count', 'ABGA_DATE', 'ABGA_pH',
       'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3', 'ABGA_O2sat', 'ABGA_FiO2',
       'ABGA_PFratio']]
raw_data_NMV = NMV[['ID', 'PID','gender','death','age','mv', 'MV_time', 'R', 'HR_DATE', 'HR',
       'RR_DATE', 'RR', 'SBP_DATE', 'SBP', 'DBP_DATE', 'DBP', 'SpO₂_DATE',
       'SpO₂', 'BT_DATE', 'BT', 'CBC_DATE', 'Hb', 'Hct', 'RBC Count', 'MCV',
       'MCH', 'MCHC', 'WBC_Count', 'PLT_Count', 'ABGA_DATE', 'ABGA_pH',
       'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3', 'ABGA_O2sat', 'ABGA_FiO2',
       'ABGA_PFratio']]
print( raw_data_MV.shape)
print( raw_data_NMV.shape)

raw_data = pd.concat([raw_data_MV,raw_data_NMV]) 
raw_data.rename(columns = {'PID':'pid'}, inplace = True)

print(len(raw_data_MV['ID'].unique()))
print(len(raw_data_NMV['ID'].unique()))
raw_data.columns

(5686, 37)
(131607, 37)
559
1587


Index(['ID', 'pid', 'gender', 'death', 'age', 'mv', 'MV_time', 'R', 'HR_DATE',
       'HR', 'RR_DATE', 'RR', 'SBP_DATE', 'SBP', 'DBP_DATE', 'DBP',
       'SpO₂_DATE', 'SpO₂', 'BT_DATE', 'BT', 'CBC_DATE', 'Hb', 'Hct',
       'RBC Count', 'MCV', 'MCH', 'MCHC', 'WBC_Count', 'PLT_Count',
       'ABGA_DATE', 'ABGA_pH', 'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3',
       'ABGA_O2sat', 'ABGA_FiO2', 'ABGA_PFratio'],
      dtype='object')

In [ ]:
static = raw_data[['pid','age','gender','mv', 'MV_time']].copy() 
static.drop_duplicates(subset=['pid'],keep='first', inplace=True)
static.head()

#Extract date in/out date
date = raw_data[['pid','HR_DATE','RR_DATE','SBP_DATE','DBP_DATE','SpO₂_DATE','BT_DATE']].copy() 
date=date.set_index('pid').dropna(how='all').reset_index()
date.head()

date_in = date.groupby('pid').first()
date_in['icu_in'] = date_in.min(axis=1)
date_in.reset_index(inplace=True)
date_in = date_in[['pid','icu_in']]

date_out = date.set_index('pid')
date_out['icu_out'] = date_out.max(axis=1)
date_out = date_out[['icu_out']].reset_index()
date_out = date_out[date_out.groupby('pid').icu_out.transform('max') == date_out['icu_out']]
date_out.drop_duplicates(inplace=True)

static  = pd.merge(left = date_out, right = static, how = "left", on='pid')
static  = pd.merge(left = date_in, right = static, how = "left", on='pid')
static['los'] = static['icu_out'] - static['icu_in']
static.loc[:, 'intime'] = static['icu_in'].apply(lambda x : x.hour)
static.head()

,pid,icu_in,icu_out,age,gender,mv,MV_time,los,intime
0,300001038,2019-12-26 16:00:00,2020-01-06 08:00:00,80,M,0,NaN,10 days 16:00:00,16
1,300001291,2019-10-21 02:00:00,2019-11-01 22:00:00,87,M,0,NaN,11 days 20:00:00,2
2,300004624,2019-08-18 10:00:00,2019-08-29 23:30:00,76,M,0,NaN,11 days 13:30:00,10
3,300006647,2019-09-08 05:00:00,2019-09-19 22:00:00,59,M,0,NaN,11 days 17:00:00,5
4,300006811,2019-10-30 22:30:00,2019-11-10 23:00:00,76,M,0,NaN,11 days 00:30:00,22


In [ ]:
#filter row with mv code but no mv time - 10 patients
static['IHMV'] = static['mv'] == 1
static['mv_code']  = ~pd.isnull(static['MV_time'])
print(static.shape)
static=static[static['mv_code']==static['IHMV']]
print(static.shape)
static = static.drop(labels='mv_code',axis=1)
static.head()

(1750, 11)
(1740, 11)


,pid,icu_in,icu_out,age,gender,mv,MV_time,los,intime,IHMV
0,300001038,2019-12-26 16:00:00,2020-01-06 08:00:00,80,M,0,NaN,10 days 16:00:00,16,False
1,300001291,2019-10-21 02:00:00,2019-11-01 22:00:00,87,M,0,NaN,11 days 20:00:00,2,False
2,300004624,2019-08-18 10:00:00,2019-08-29 23:30:00,76,M,0,NaN,11 days 13:30:00,10,False
3,300006647,2019-09-08 05:00:00,2019-09-19 22:00:00,59,M,0,NaN,11 days 17:00:00,5,False
4,300006811,2019-10-30 22:30:00,2019-11-10 23:00:00,76,M,0,NaN,11 days 00:30:00,22,False


In [ ]:
#get time-to-MV column
t=~pd.isnull(static['MV_time'])
static['MV_time_'] = np.nan
static['MV_time'] = static['MV_time'].astype('datetime64')
static['MV_time_'][t] = (static['MV_time'][t] - static['icu_in'][t]).astype('timedelta64[h]')
static['MV_time_'][t]
static.head()

<ipython-input-5-9cc07e1d29a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static['MV_time_'][t] = (static['MV_time'][t] - static['icu_in'][t]).astype('timedelta64[h]')


,pid,icu_in,icu_out,age,gender,mv,MV_time,los,intime,IHMV,MV_time_
0,300001038,2019-12-26 16:00:00,2020-01-06 08:00:00,80,M,0,NaT,10 days 16:00:00,16,False,NaN
1,300001291,2019-10-21 02:00:00,2019-11-01 22:00:00,87,M,0,NaT,11 days 20:00:00,2,False,NaN
2,300004624,2019-08-18 10:00:00,2019-08-29 23:30:00,76,M,0,NaT,11 days 13:30:00,10,False,NaN
3,300006647,2019-09-08 05:00:00,2019-09-19 22:00:00,59,M,0,NaT,11 days 17:00:00,5,False,NaN
4,300006811,2019-10-30 22:30:00,2019-11-10 23:00:00,76,M,0,NaT,11 days 00:30:00,22,False,NaN


In [ ]:
static_to_keep = static[['pid','age','icu_in','intime','gender']].copy()
static_to_keep = pd.get_dummies(static_to_keep, columns = ['gender'])
static_to_keep.sort_values(by=['pid'],inplace = True)
static_to_keep['pid']=static_to_keep['pid'].round(0).astype(int)
static_to_keep.head()

,pid,age,icu_in,intime,gender_F,gender_M
0,300001038,80,2019-12-26 16:00:00,16,0,1
1,300001291,87,2019-10-21 02:00:00,2,0,1
2,300004624,76,2019-08-18 10:00:00,10,0,1
3,300006647,59,2019-09-08 05:00:00,5,0,1
4,300006811,76,2019-10-30 22:30:00,22,0,1


In [ ]:
vital=raw_data[['pid', 'HR_DATE', 'HR',
       'RR_DATE', 'RR', 'SBP_DATE', 'SBP', 'DBP_DATE', 'DBP', 'SpO₂_DATE',
       'SpO₂', 'BT_DATE', 'BT']]
ABGA=raw_data[['pid',  'ABGA_DATE', 'ABGA_pH',
       'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3', 'ABGA_O2sat', 'ABGA_FiO2',
       'ABGA_PFratio']]
CBC=raw_data[['pid', 'CBC_DATE', 'Hb', 'Hct', 'RBC Count', 'MCV',
       'MCH', 'MCHC', 'WBC_Count', 'PLT_Count' ]]
CBC = CBC.set_index('pid').dropna(how='all').reset_index()
ABGA = ABGA.set_index('pid').dropna(how='all').reset_index()
vital = vital.set_index('pid').dropna(how='all').reset_index()

print(vital.columns)
print(CBC.columns) 
print(ABGA.columns)

_vital = ['rr','hr','spo2','sbp','dbp','temp']
_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']
cols = _vital+_CBC+_ABGA
ID_COLS = ['pid','date']
vital.head()

Index(['pid', 'HR_DATE', 'HR', 'RR_DATE', 'RR', 'SBP_DATE', 'SBP', 'DBP_DATE',
       'DBP', 'SpO₂_DATE', 'SpO₂', 'BT_DATE', 'BT'],
      dtype='object')
Index(['pid', 'CBC_DATE', 'Hb', 'Hct', 'RBC Count', 'MCV', 'MCH', 'MCHC',
       'WBC_Count', 'PLT_Count'],
      dtype='object')
Index(['pid', 'ABGA_DATE', 'ABGA_pH', 'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3',
       'ABGA_O2sat', 'ABGA_FiO2', 'ABGA_PFratio'],
      dtype='object')


,pid,HR_DATE,HR,RR_DATE,RR,SBP_DATE,SBP,DBP_DATE,DBP,SpO₂_DATE,SpO₂,BT_DATE,BT
0,300014389,2019-11-20 22:00:00,84.0,2019-11-20 22:00:00,21.0,2019-11-20 22:00:00,125.0,2019-11-20 22:00:00,73.0,2019-11-20 22:00:00,100.0,2019-11-20 22:00:00,36.4
1,300014389,2019-11-20 23:00:00,68.0,2019-11-20 23:00:00,19.0,2019-11-20 23:00:00,129.0,2019-11-20 23:00:00,71.0,2019-11-20 23:00:00,100.0,2019-11-20 23:00:00,36.4
2,300014389,2019-11-21 00:00:00,74.0,2019-11-21 00:00:00,22.0,2019-11-21 00:00:00,145.0,2019-11-21 00:00:00,74.0,2019-11-21 00:00:00,100.0,2019-11-21 00:00:00,35.7
3,300014389,2019-11-21 01:00:00,72.0,2019-11-21 01:00:00,20.0,2019-11-21 01:00:00,133.0,2019-11-21 01:00:00,72.0,2019-11-21 01:00:00,100.0,2019-11-21 01:00:00,36.0
4,300014389,2019-11-21 02:00:00,66.0,2019-11-21 02:00:00,18.0,2019-11-21 02:00:00,145.0,2019-11-21 02:00:00,70.0,2019-11-21 02:00:00,100.0,2019-11-21 02:00:00,36.1


In [ ]:
#vital preprocessing
#_vital = ['rr','hr','spo2','sbp','dbp','temp']

hr = vital.iloc[:,[0,1,2]].copy()
rr = vital.iloc[:,[0,3,4]].copy()
sbp = vital.iloc[:,[0,5,6]].copy()
dbp = vital.iloc[:,[0,7,8]].copy()
spo2 = vital.iloc[:,[0,9,10]].copy()
bt = vital.iloc[:,[0,11,12]].copy()

vital = [sbp,dbp,bt,rr,hr,spo2]

for i, vt in enumerate(vital):
    vt.rename(columns = { i: 'date' for i in  vt.columns if i.endswith('DATE') }, inplace=True )
    vt.sort_values(by=['pid','date'],inplace=True)
    #vt = vt.set_index('pid')
    print(vt.shape)
    vt = vt.dropna(subset=['date'],how='any')
    print(vt.shape)
    vital[i] = vt

print('do merge')
vital_icu = None
for i, vt in enumerate(vital):
    if i == 0:
        vital_icu=vt
    else:
        vital_icu = pd.merge(left = vital_icu, right = vt, how = "outer", on=['pid','date'])
        
vital_icu.sort_values(by=['pid','date'],inplace=True)
vital_icu.columns =  ['pid','date','sbp','dbp','temp','rr','hr','spo2']
vital_icu=vital_icu[ID_COLS+_vital]
vital_icu.head()

(135451, 3)
(125663, 3)
(135451, 3)
(125651, 3)
(135451, 3)
(105877, 3)
(135451, 3)
(128764, 3)
(135451, 3)
(129416, 3)
(135451, 3)
(111110, 3)
do merge


,pid,date,rr,hr,spo2,sbp,dbp,temp
0,300001038,2019-12-26 16:00:00,12.0,45.0,98.0,99.0,46.0,NaN
1,300001038,2019-12-26 17:00:00,14.0,49.0,99.0,113.0,51.0,36.0
2,300001038,2019-12-26 18:00:00,16.0,57.0,97.0,120.0,59.0,36.3
3,300001038,2019-12-26 19:00:00,17.0,53.0,96.0,108.0,62.0,NaN
4,300001038,2019-12-26 20:00:00,15.0,55.0,95.0,100.0,44.0,36.5


In [ ]:
# CBC processing
#_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
print(CBC.shape)
CBC.rename(columns = {'CBC_DATE':'date'}, inplace = True)
#CBC['date']=pd.to_datetime(CBC['date'])
#CBC.dropna(subset=['pid'],inplace=True)
CBC.dropna(subset=['date'],inplace=True)
CBC.sort_values(by=['pid','date'],inplace=True)
#CBC=CBC[ID_COLS + _CBC]
CBC.columns = ['pid', 'date', 'hb', 'hct', 'rbc', 'mcv', 'mch', 'mchc',
       'wbc', 'platelets']
CBC=CBC[ID_COLS + _CBC]
CBC.describe()

(10300, 10)


,pid,hct,hb,platelets,wbc,rbc,mch,mchc,mcv
count,1.029900e+04,10299.000000,10299.000000,10299.000000,10299.000000,10289.000000,10299.000000,10299.000000,10299.000000
mean,3.012974e+08,31.799252,10.573085,187.500728,9.909853,3.378511,31.438518,33.242616,94.537110
std,5.627126e+05,6.941302,2.333525,98.312119,7.572762,0.770918,2.380986,0.958456,6.156225
min,3.000010e+08,5.400000,1.500000,3.000000,0.100000,0.510000,19.300000,27.200000,67.300000
25%,3.008539e+08,26.500000,8.800000,123.000000,6.300000,2.800000,30.100000,32.700000,90.900000
50%,3.014999e+08,31.300000,10.300000,178.000000,8.540000,3.330000,31.400000,33.300000,94.100000
75%,3.018102e+08,36.800000,12.300000,237.000000,11.855000,3.930000,32.700000,33.800000,97.900000
max,3.018377e+08,63.600000,19.300000,1030.000000,322.600000,8.690000,47.600000,37.800000,150.200000


In [ ]:
# ABGA processing
#_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']
cols = _vital+_CBC+_ABGA
print(ABGA.shape)
print(ABGA.columns)
ABGA.rename(columns = {'ABGA_DATE':'date'}, inplace = True)
ABGA.dropna(subset=['date'],inplace=True)
ABGA.sort_values(by=['pid','date'],inplace=True)
ABGA.columns = ['pid', 'date', 'ph',  'pco2', 'po2', 'hco3', 'sao2', 'fio2', 'pfratio']
ABGA=ABGA[ID_COLS + _ABGA]
ABGA.head()

(5930, 9)
Index(['pid', 'ABGA_DATE', 'ABGA_pH', 'ABGA_PCO2', 'ABGA_PO2', 'ABGA_HCO3',
       'ABGA_O2sat', 'ABGA_FiO2', 'ABGA_PFratio'],
      dtype='object')


,pid,date,ph,hco3,po2,pco2,sao2,fio2,pfratio
1152,300001291,2019-10-21 05:02:58,7.434,28.5,85.8,42.1,96.8,20.9,410.6
1153,300001291,2019-10-21 09:01:28,7.424,30.2,103.9,45.7,98.0,30.0,346.3
1154,300001291,2019-10-21 15:35:07,7.475,28.1,182.9,37.8,99.7,32.0,571.4
1155,300001291,2019-10-22 06:19:58,7.457,28.5,73.7,40.0,95.4,20.9,352.7
1156,300004624,2019-08-19 05:31:11,7.448,20.2,107.1,29.0,98.5,20.9,512.3


In [ ]:
#merge lab & vital data
X = pd.merge(left = vital_icu, right = CBC, how = "outer", on=['pid','date'])
X = pd.merge(left = X, right = ABGA, how = "outer", on=['pid','date'])
X.sort_values(by=['pid','date'],inplace=True)
X=X.reset_index(drop=True)
X = pd.merge(left = static_to_keep[['pid','icu_in']], right = X, how = "inner", on='pid')

#date to hours_in
X.insert(1,'hours_in','')
X['hours_in'] = (X['date'] - X['icu_in']).astype('timedelta64[h]')
X.drop(['icu_in','date'], axis=1, inplace=True)
X.head()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
0,300001038,-5.0,NaN,NaN,NaN,NaN,NaN,NaN,38.1,12.5,...,30.5,32.8,92.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,300001038,0.0,12.0,45.0,98.0,99.0,46.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,300001038,1.0,14.0,49.0,99.0,113.0,51.0,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,300001038,2.0,16.0,57.0,97.0,120.0,59.0,36.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,300001038,3.0,17.0,53.0,96.0,108.0,62.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# save raw data (no imputation & no outlier removal)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

X_raw= X.copy()
X_raw[_vital].boxplot()

<AxesSubplot:>

In [ ]:
#delete outlier

#_vital = ['rr','hr','spo2','sbp','dbp','temp']
#_CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
#_ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']
#cols = _vital+_CBC+_ABGA

outlier_range = {}
outlier_range['rr']=[0,375]
outlier_range['hr']=[0,390]
outlier_range['spo2']=[0,150]
outlier_range['sbp']=[0,375]
outlier_range['dbp']=[0,375]
outlier_range['temp']=[14.2,47]

outlier_range['hct']=[0,100]
outlier_range['hb']=[0,30]
outlier_range['platelets']=[0,2200]
outlier_range['wbc']=[0,1100]

outlier_range['ph']=[0,10]
outlier_range['po2']=[0,770]
outlier_range['pco2']=[0,220]
outlier_range['sao2']=[0,150]
outlier_range['fio2']=[20,110]

for i in outlier_range.keys():
    X.mask(X[i]<outlier_range[i][0], inplace = True)
    X.mask(X[i]>outlier_range[i][1], inplace = True)
# change fio2 scale
X['fio2'] = X['fio2']/100
X.describe()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,1.579260e+05,157926.000000,129290.000000,129947.000000,111701.000000,125687.000000,125676.000000,106347.000000,10115.000000,10115.000000,...,10115.000000,10115.000000,10115.000000,5852.000000,5845.000000,5862.000000,5863.000000,5837.000000,5634.000000,5635.000000
mean,3.013101e+08,135.647873,18.844466,82.218428,97.713319,125.735788,73.931968,36.951184,31.750588,10.561354,...,31.440415,33.255136,94.505467,7.394155,21.640975,124.295172,35.398414,96.772349,0.331150,447.233274
std,5.534240e+05,368.722030,4.756054,17.227449,2.806472,19.469352,14.072787,0.499463,6.910072,2.326444,...,2.383141,0.949212,6.149531,0.078771,5.238685,64.937956,10.796019,4.315254,0.174507,291.608062
min,3.000010e+08,-1069.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.700000,5.400000,1.500000,...,19.300000,27.200000,67.300000,6.904000,4.100000,19.800000,6.800000,28.700000,0.200000,34.800000
25%,3.008754e+08,21.000000,16.000000,70.000000,96.000000,112.000000,65.000000,36.600000,26.500000,8.800000,...,30.100000,32.700000,90.900000,7.350000,18.800000,78.900000,28.800000,95.800000,0.209000,258.700000
50%,3.015201e+08,52.000000,18.000000,80.000000,98.000000,125.000000,73.000000,36.900000,31.200000,10.300000,...,31.400000,33.300000,94.100000,7.404000,21.600000,101.300000,33.700000,97.900000,0.240000,387.700000
75%,3.018104e+08,124.000000,21.000000,92.000000,100.000000,139.000000,82.000000,37.300000,36.800000,12.300000,...,32.700000,33.800000,97.800000,7.448000,24.300000,153.175000,40.100000,99.300000,0.400000,534.250000
max,3.018377e+08,4289.000000,250.000000,220.000000,100.000000,369.000000,367.000000,42.000000,63.600000,19.300000,...,47.600000,37.800000,150.200000,7.845000,52.700000,528.400000,200.000000,100.000000,1.000000,2528.100000


In [ ]:
#print statistics
temp = X.copy()
temp = temp[temp['hours_in']>=0] # remove observation before ICU admission
temp = temp[temp['hours_in']<=240] # maximum, 10 days of stay
temp=temp.groupby('pid').mean()
temp.mean()

hours_in      47.334824
rr            18.495208
hr            80.675169
spo2          97.448521
sbp          124.599622
dbp           74.232704
temp          36.866075
hct           34.110718
hb            11.387406
platelets    187.060154
wbc            9.407532
rbc            3.623776
mch           31.547275
mchc          33.368075
mcv           94.514292
ph             7.391682
hco3          21.807389
po2          133.901051
pco2          35.983293
sao2          97.335806
fio2           0.306104
pfratio      500.131161
dtype: float64

In [ ]:
X.describe()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,1.579260e+05,157926.000000,129290.000000,129947.000000,111701.000000,125687.000000,125676.000000,106347.000000,10115.000000,10115.000000,...,10115.000000,10115.000000,10115.000000,5852.000000,5845.000000,5862.000000,5863.000000,5837.000000,5634.000000,5635.000000
mean,3.013101e+08,135.647873,18.844466,82.218428,97.713319,125.735788,73.931968,36.951184,31.750588,10.561354,...,31.440415,33.255136,94.505467,7.394155,21.640975,124.295172,35.398414,96.772349,0.331150,447.233274
std,5.534240e+05,368.722030,4.756054,17.227449,2.806472,19.469352,14.072787,0.499463,6.910072,2.326444,...,2.383141,0.949212,6.149531,0.078771,5.238685,64.937956,10.796019,4.315254,0.174507,291.608062
min,3.000010e+08,-1069.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.700000,5.400000,1.500000,...,19.300000,27.200000,67.300000,6.904000,4.100000,19.800000,6.800000,28.700000,0.200000,34.800000
25%,3.008754e+08,21.000000,16.000000,70.000000,96.000000,112.000000,65.000000,36.600000,26.500000,8.800000,...,30.100000,32.700000,90.900000,7.350000,18.800000,78.900000,28.800000,95.800000,0.209000,258.700000
50%,3.015201e+08,52.000000,18.000000,80.000000,98.000000,125.000000,73.000000,36.900000,31.200000,10.300000,...,31.400000,33.300000,94.100000,7.404000,21.600000,101.300000,33.700000,97.900000,0.240000,387.700000
75%,3.018104e+08,124.000000,21.000000,92.000000,100.000000,139.000000,82.000000,37.300000,36.800000,12.300000,...,32.700000,33.800000,97.800000,7.448000,24.300000,153.175000,40.100000,99.300000,0.400000,534.250000
max,3.018377e+08,4289.000000,250.000000,220.000000,100.000000,369.000000,367.000000,42.000000,63.600000,19.300000,...,47.600000,37.800000,150.200000,7.845000,52.700000,528.400000,200.000000,100.000000,1.000000,2528.100000


In [ ]:
reference_value = {      # average
#vital
    'rr': 16,           # 19.75339
    'hr':80,            # 86.15997
    'spo2': 97,      # 90.743683
    'sbp':110,          # 122.7326
    'dbp':70,           # 60.76322
    'temp':36.5,        # 37.0867
#CBC
    'hct': 41.5,        # 30.491288
    'hb': 13.5,         # 10.396213
    'platelets':275,    # 219.158934
    'wbc': 7,           # 12.412393
    'rbc': 4.45,        # 3.454153
    'mch': 29.5,        # 30.333704
    'mchc': 33.25,      # 33.850937
    'mcv': 90,          # 89.702025
#ABGA
    'ph': 7.4,          # 7.388153
    'hco3': 24,         # 24.595758
    'po2': 87.5,        # 130.492106
    'pco2': 40,         # 41.616208
    'sao2': 97,     # 97.09498
    'fio2' : 0.21       # 0.512136
}

#extract aggregate value per hour
X_ = X.groupby(['pid'] + ['hours_in']).agg(['mean', 'first','min','max'])
print(X.columns)

if DROPREF:
    print("DROPREF")
    print(X_.shape)
    print(X_.isna().sum().sum())
    X_=X_.groupby(['pid'] ).fillna(method='ffill')
    print(X_.isna().sum().sum())
    X_=X_.dropna(axis=0,thresh=23-5)
    print(X_.isna().sum().sum())
    print(X_.shape)
else:
    X_=X_.groupby(['pid'] ).fillna(method='ffill').groupby(['pid'] ).fillna(method='bfill')
#X_=X_.groupby(['pid'] ).fillna(method='ffill').groupby(['pid'] ).fillna(method='bfill',limit=6)
#X_=X_.groupby(['pid'] ).fillna(method='ffill')

print(X_.isna().sum().sum())
X_.fillna(
    pd.Series(X_.columns.get_level_values(0).map(reference_value.get), X_.columns), inplace=True
)
X_.head()

Index(['pid', 'hours_in', 'rr', 'hr', 'spo2', 'sbp', 'dbp', 'temp', 'hct',
       'hb', 'platelets', 'wbc', 'rbc', 'mch', 'mchc', 'mcv', 'ph', 'hco3',
       'po2', 'pco2', 'sao2', 'fio2', 'pfratio'],
      dtype='object')
DROPREF
(137797, 84)
8322700
1591112
1580212
(137658, 84)
1580212


rr                      hr                    spo2  \
                      mean first   min   max  mean first   min   max  mean   
pid         hours_in                                                         
300001038.0 -5.0      16.0  16.0  16.0  16.0  80.0  80.0  80.0  80.0  97.0   
             0.0      12.0  12.0  12.0  12.0  45.0  45.0  45.0  45.0  98.0   
             1.0      14.0  14.0  14.0  14.0  49.0  49.0  49.0  49.0  99.0   
             2.0      16.0  16.0  16.0  16.0  57.0  57.0  57.0  57.0  97.0   
             3.0      17.0  17.0  17.0  17.0  53.0  53.0  53.0  53.0  96.0   

                            ...  sao2        fio2                   pfratio  \
                     first  ...   min   max  mean first   min   max    mean   
pid         hours_in        ...                                               
300001038.0 -5.0      97.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
             0.0      98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
             1.0      99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
             2.0      97.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
             3.0      96.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   

                                    
                     first min max  
pid         hours_in                
300001038.0 -5.0       NaN NaN NaN  
             0.0       NaN NaN NaN  
             1.0       NaN NaN NaN  
             2.0       NaN NaN NaN  
             3.0       NaN NaN NaN  

[5 rows x 84 columns]

In [ ]:
X_=X_.reset_index()
X_ = X_[X_['hours_in']>=0] # remove observation before ICU admission
X_ = X_[X_['hours_in']<=240] # maximum, 10 days of stay

# filter patients with earlyexit ==> 7 patients removed
temp = X_
print(len(temp['pid'].reset_index(drop=True).unique()))
temp = temp.reset_index(drop=True).groupby('pid').filter(lambda x: (x.hours_in >= 6).any() )
print(len(temp['pid'].unique()))
X_ = temp 
X_ = X_.set_index(['pid','hours_in'])
X_.head()

1740
1695


rr                      hr                    spo2  \
                      mean first   min   max  mean first   min   max  mean   
pid         hours_in                                                         
300001038.0 0.0       12.0  12.0  12.0  12.0  45.0  45.0  45.0  45.0  98.0   
            1.0       14.0  14.0  14.0  14.0  49.0  49.0  49.0  49.0  99.0   
            2.0       16.0  16.0  16.0  16.0  57.0  57.0  57.0  57.0  97.0   
            3.0       17.0  17.0  17.0  17.0  53.0  53.0  53.0  53.0  96.0   
            4.0       15.0  15.0  15.0  15.0  55.0  55.0  55.0  55.0  95.0   

                            ...  sao2        fio2                   pfratio  \
                     first  ...   min   max  mean first   min   max    mean   
pid         hours_in        ...                                               
300001038.0 0.0       98.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            1.0       99.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            2.0       97.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            3.0       96.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   
            4.0       95.0  ...  97.0  97.0  0.21  0.21  0.21  0.21     NaN   

                                    
                     first min max  
pid         hours_in                
300001038.0 0.0        NaN NaN NaN  
            1.0        NaN NaN NaN  
            2.0        NaN NaN NaN  
            3.0        NaN NaN NaN  
            4.0        NaN NaN NaN  

[5 rows x 84 columns]

In [ ]:
idx = pd.IndexSlice
X_worst = X_.loc[:,idx[:, ['min','max']]].copy()
cnt=0
for k in X_worst.columns:
    if k[0] == 'pfratio':
        X_worst[k[0],'worst'] = X_worst[k[0],'min']
        continue
    #for min, max calc distance from reference value
    cnt=cnt+1
    if cnt%2==0:
        continue
    
    ref = reference_value[k[0]]
    print(k,cnt,ref)
    min_diff = abs(X_worst[k[0],'min'] - reference_value[k[0]] )
    max_diff = abs(X_worst[k[0],'max'] - reference_value[k[0]] )
    
    X_worst[k[0],'worst'] = np.where(max_diff>min_diff,
                    X_worst[k[0],'max'], X_worst[k[0],'min'])

X_worst = X_worst.loc[:,idx[:, ['worst']]]
X_worst.columns = X_worst.columns.droplevel(-1)
X_worst.head()

('rr', 'min') 1 16
('hr', 'min') 3 80
('spo2', 'min') 5 97
('sbp', 'min') 7 110
('dbp', 'min') 9 70
('temp', 'min') 11 36.5
('hct', 'min') 13 41.5
('hb', 'min') 15 13.5
('platelets', 'min') 17 275
('wbc', 'min') 19 7
('rbc', 'min') 21 4.45
('mch', 'min') 23 29.5
('mchc', 'min') 25 33.25
('mcv', 'min') 27 90
('ph', 'min') 29 7.4
('hco3', 'min') 31 24
('po2', 'min') 33 87.5
('pco2', 'min') 35 40
('sao2', 'min') 37 97
('fio2', 'min') 39 0.21


rr    hr  spo2    sbp   dbp  temp   hct    hb  \
pid         hours_in                                                    
300001038.0 0.0       12.0  45.0  98.0   99.0  46.0  36.5  38.1  12.5   
            1.0       14.0  49.0  99.0  113.0  51.0  36.0  38.1  12.5   
            2.0       16.0  57.0  97.0  120.0  59.0  36.3  38.1  12.5   
            3.0       17.0  53.0  96.0  108.0  62.0  36.3  38.1  12.5   
            4.0       15.0  55.0  95.0  100.0  44.0  36.5  38.1  12.5   

                      platelets   wbc  ...   mch  mchc   mcv   ph  hco3   po2  \
pid         hours_in                   ...                                      
300001038.0 0.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            1.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            2.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            3.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            4.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   

                      pco2  sao2  fio2  pfratio  
pid         hours_in                             
300001038.0 0.0       40.0  97.0  0.21      NaN  
            1.0       40.0  97.0  0.21      NaN  
            2.0       40.0  97.0  0.21      NaN  
            3.0       40.0  97.0  0.21      NaN  
            4.0       40.0  97.0  0.21      NaN  

[5 rows x 21 columns]

In [ ]:
#split dataset --> mean / first / worst
def split_data(X,target):
    idx = pd.IndexSlice
    X_=X.loc[:,idx[cols, target]]
    X_.columns = X_.columns.droplevel(-1)
    X_.columns.name = None
    return X_
X_mean = split_data(X_,'mean')
X_first = split_data(X_,'first')

dataset = [X_worst,X_mean,X_first]

In [ ]:
static_to_keep.drop(columns='icu_in',inplace=True)


In [ ]:
#fillin pfratio
for idx, df in enumerate(dataset):
    # fillin pfratio
    df['pfratio'] = np.where(df.pfratio.isnull(), df.po2/df.fio2, df.pfratio )
    dataset[idx]=df
dataset[0].head()

<ipython-input-21-cd4118560be6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pfratio'] = np.where(df.pfratio.isnull(), df.po2/df.fio2, df.pfratio )


rr    hr  spo2    sbp   dbp  temp   hct    hb  \
pid         hours_in                                                    
300001038.0 0.0       12.0  45.0  98.0   99.0  46.0  36.5  38.1  12.5   
            1.0       14.0  49.0  99.0  113.0  51.0  36.0  38.1  12.5   
            2.0       16.0  57.0  97.0  120.0  59.0  36.3  38.1  12.5   
            3.0       17.0  53.0  96.0  108.0  62.0  36.3  38.1  12.5   
            4.0       15.0  55.0  95.0  100.0  44.0  36.5  38.1  12.5   

                      platelets   wbc  ...   mch  mchc   mcv   ph  hco3   po2  \
pid         hours_in                   ...                                      
300001038.0 0.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            1.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            2.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            3.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   
            4.0           127.0  6.07  ...  30.5  32.8  92.9  7.4  24.0  87.5   

                      pco2  sao2  fio2     pfratio  
pid         hours_in                                
300001038.0 0.0       40.0  97.0  0.21  416.666667  
            1.0       40.0  97.0  0.21  416.666667  
            2.0       40.0  97.0  0.21  416.666667  
            3.0       40.0  97.0  0.21  416.666667  
            4.0       40.0  97.0  0.21  416.666667  

[5 rows x 21 columns]

In [ ]:
#apply merge
for idx, df in enumerate(dataset):
    dataset[idx]= pd.merge(df.reset_index(), static_to_keep, on='pid')
dataset[0].head()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,hco3,po2,pco2,sao2,fio2,pfratio,age,intime,gender_F,gender_M
0,300001038.0,0.0,12.0,45.0,98.0,99.0,46.0,36.5,38.1,12.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,80,16,0,1
1,300001038.0,1.0,14.0,49.0,99.0,113.0,51.0,36.0,38.1,12.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,80,16,0,1
2,300001038.0,2.0,16.0,57.0,97.0,120.0,59.0,36.3,38.1,12.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,80,16,0,1
3,300001038.0,3.0,17.0,53.0,96.0,108.0,62.0,36.3,38.1,12.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,80,16,0,1
4,300001038.0,4.0,15.0,55.0,95.0,100.0,44.0,36.5,38.1,12.5,...,24.0,87.5,40.0,97.0,0.21,416.666667,80,16,0,1


In [ ]:
# add absolute time feature
for idx, df in enumerate(dataset):
    abs_time = (df['intime'] + df['hours_in'])%24
    df.insert(2, 'absolute_time', abs_time)
    df.drop('intime', axis=1, inplace=True)
    dataset[idx]=df
dataset[0].head()

,pid,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,300001038.0,0.0,16.0,12.0,45.0,98.0,99.0,46.0,36.5,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
1,300001038.0,1.0,17.0,14.0,49.0,99.0,113.0,51.0,36.0,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
2,300001038.0,2.0,18.0,16.0,57.0,97.0,120.0,59.0,36.3,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
3,300001038.0,3.0,19.0,17.0,53.0,96.0,108.0,62.0,36.3,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
4,300001038.0,4.0,20.0,15.0,55.0,95.0,100.0,44.0,36.5,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1


In [ ]:
# remove static_covid with no data(X)
print(len(dataset[0]['pid'].unique()))
print(len(static_to_keep['pid'].unique()))
print(len(static['pid'].unique()))

pid_list = dataset[0]['pid'].unique()
static_to_keep=static_to_keep[static_to_keep['pid'].isin(pid_list)]
static=static[static['pid'].isin(pid_list)]


1695
1740
1740


In [ ]:
#filter observation after MV
for idx, df in enumerate(dataset):
    for subject_id in static[static['IHMV']==True]['pid']:
        vent_time = static[static['pid']==subject_id]['MV_time_'].item()
        df = df.drop(df[(df['pid']==subject_id) & (df['hours_in'] >= vent_time)].index)
    dataset[idx]=df
dataset[0].head()

,pid,hours_in,absolute_time,rr,hr,spo2,sbp,dbp,temp,hct,...,ph,hco3,po2,pco2,sao2,fio2,pfratio,age,gender_F,gender_M
0,300001038.0,0.0,16.0,12.0,45.0,98.0,99.0,46.0,36.5,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
1,300001038.0,1.0,17.0,14.0,49.0,99.0,113.0,51.0,36.0,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
2,300001038.0,2.0,18.0,16.0,57.0,97.0,120.0,59.0,36.3,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
3,300001038.0,3.0,19.0,17.0,53.0,96.0,108.0,62.0,36.3,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1
4,300001038.0,4.0,20.0,15.0,55.0,95.0,100.0,44.0,36.5,38.1,...,7.4,24.0,87.5,40.0,97.0,0.21,416.666667,80,0,1


In [ ]:
# extract Y label
Y=dataset[0][['pid','hours_in']].copy()
Y['vent'] = 0

for subject_id in static[static['IHMV']==True]['pid']:
    vent_time = static[static['pid']==subject_id]['MV_time_'].item()
    idx = Y[(Y['pid']==subject_id) & (Y['hours_in'] >= vent_time-12)].index
    Y.loc[idx, 'vent'] = 1
    # print(Y.loc[idx, 'vent'])
Y.set_index(['pid','hours_in'],inplace=True)
Y.describe()

,vent
count,124829.000000
mean,0.010727
std,0.103013
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
#set index
for idx, df in enumerate(dataset):
    df.set_index(['pid','hours_in'],inplace=True)
    dataset[idx]=df
dataset[0].columns

Index(['absolute_time', 'rr', 'hr', 'spo2', 'sbp', 'dbp', 'temp', 'hct', 'hb',
       'platelets', 'wbc', 'rbc', 'mch', 'mchc', 'mcv', 'ph', 'hco3', 'po2',
       'pco2', 'sao2', 'fio2', 'pfratio', 'age', 'gender_F', 'gender_M'],
      dtype='object')

In [ ]:
def id_cols(df):
    df.rename(columns = {'pid':'subject_id'}, inplace = True)
    df.insert(1,'hadm_id', 0 )
    df.insert(2,'icustay_id', 0 )
    return df

for idx, df in enumerate(dataset):
    dataset[idx]=id_cols(df.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
Y=id_cols(Y.reset_index()).set_index(['subject_id', 'hadm_id', 'icustay_id','hours_in'])
static=id_cols(static)
static_to_keep=id_cols(static_to_keep)

In [ ]:
vital = ['rr','hr','spo2','sbp','dbp','temp']
CBC = ['hct','hb','platelets','wbc','rbc','mch','mchc','mcv']
ABGA = ['ph','hco3', 'po2','pco2','sao2','fio2','pfratio']

dataset[0].reset_index()[ABGA].describe()

,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,124829.000000,124829.000000,124829.000000,124829.000000,124829.000000,124829.000000,124829.000000
mean,7.410127,22.570396,107.866398,36.000608,97.170381,0.261754,444.093954
std,0.048573,3.496464,44.318157,7.222635,2.934337,0.111784,189.800299
min,6.904000,4.100000,19.800000,7.600000,28.700000,0.200000,34.800000
25%,7.400000,20.600000,87.500000,30.700000,97.000000,0.209000,366.400000
50%,7.400000,24.000000,87.500000,38.000000,97.000000,0.210000,416.666667
75%,7.436000,24.000000,117.200000,40.000000,98.700000,0.280000,466.400000
max,7.622000,52.700000,528.400000,200.000000,100.000000,1.000000,2528.100000


In [ ]:
static = static[['subject_id', 'hadm_id', 'icustay_id', 'icu_in', 'icu_out', 'age',
       'gender', 'MV_time', 'MV_time_', 'los', 'intime', 'IHMV']]

In [ ]:
static["MV_ICU"] = static["IHMV"].astype(int)

In [ ]:
import os.path

if DROPREF == False:    
    DATAFILE = "../data/ku-icu/"
    name = ['ku_rICU_worst.h5','ku_rICU_mean.h5','ku_rICU_first.h5']
    for idx, df in enumerate(dataset):
        path = os.path.join(DATAFILE,name[idx])
        print(path)
        df.to_hdf(path,'vitals_labs')
        Y.to_hdf(path,'interventions')
        static.to_hdf(path,'patients', format='table')
        static_to_keep.to_hdf(path,'patients_to_keep', format='table')
else:
    
    DATAFILE = "../data/ku-icu/dropref/"
    name = ['ku_rICU_worst.h5','ku_rICU_mean.h5','ku_rICU_first.h5']
    for idx, df in enumerate(dataset):
        path = os.path.join(DATAFILE,name[idx])
        print(path)
        df.to_hdf(path,'vitals_labs')
        Y.to_hdf(path,'interventions')
        static.to_hdf(path,'patients', format='table')
        static_to_keep.to_hdf(path,'patients_to_keep', format='table')

../data/ku-icu/dropref/ku_rICU_worst.h5
../data/ku-icu/dropref/ku_rICU_mean.h5
../data/ku-icu/dropref/ku_rICU_first.h5


In [ ]:
DATAFILE = "../data/ku-icu/ku_rICU_raw.h5"
print(DATAFILE)
X_raw.to_hdf(DATAFILE,'vitals_labs')
Y.to_hdf(DATAFILE,'interventions')
static.to_hdf(DATAFILE,'patients', format='table')
static_to_keep.to_hdf(DATAFILE,'patients_to_keep', format='table')
X_raw.describe()

../data/ku-icu/ku_rICU_raw.h5


,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,1.579280e+05,157928.000000,129292.000000,129949.000000,111702.000000,125689.000000,125678.000000,106349.000000,10115.000000,10115.000000,...,10115.000000,10115.000000,10115.000000,5852.000000,5845.000000,5862.000000,5863.000000,5837.000000,5634.000000,5635.000000
mean,3.013101e+08,135.648156,18.844422,82.221433,97.713313,125.735904,73.938247,36.951187,31.750588,10.561354,...,31.440415,33.255136,94.505467,7.394155,21.640975,124.295172,35.398414,96.772349,33.115016,447.233274
std,5.534221e+05,368.719825,4.756056,17.262452,2.806460,19.469239,14.241973,0.499468,6.910072,2.326444,...,2.383141,0.949212,6.149531,0.078771,5.238685,64.937956,10.796019,4.315254,17.450682,291.608062
min,3.000010e+08,-1069.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.700000,5.400000,1.500000,...,19.300000,27.200000,67.300000,6.904000,4.100000,19.800000,6.800000,28.700000,20.000000,34.800000
25%,3.008754e+08,21.000000,16.000000,70.000000,96.000000,112.000000,65.000000,36.600000,26.500000,8.800000,...,30.100000,32.700000,90.900000,7.350000,18.800000,78.900000,28.800000,95.800000,20.900000,258.700000
50%,3.015201e+08,52.000000,18.000000,80.000000,98.000000,125.000000,73.000000,36.900000,31.200000,10.300000,...,31.400000,33.300000,94.100000,7.404000,21.600000,101.300000,33.700000,97.900000,24.000000,387.700000
75%,3.018104e+08,124.000000,21.000000,92.000000,100.000000,139.000000,82.000000,37.300000,36.800000,12.300000,...,32.700000,33.800000,97.800000,7.448000,24.300000,153.175000,40.100000,99.300000,40.000000,534.250000
max,3.018377e+08,4289.000000,250.000000,479.000000,100.000000,369.000000,850.000000,42.000000,63.600000,19.300000,...,47.600000,37.800000,150.200000,7.845000,52.700000,528.400000,200.000000,100.000000,100.000000,2528.100000


In [ ]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATAFILE_test = "../data/ku-icu/ku_rICU_raw.h5"
X_test = pd.read_hdf(DATAFILE_test,'vitals_labs')
Y_test = pd.read_hdf(DATAFILE_test,'interventions')
static_test = pd.read_hdf(DATAFILE_test,'patients')
static_to_keep_test = pd.read_hdf(DATAFILE_test,'patients_to_keep')
X_test.describe()

,pid,hours_in,rr,hr,spo2,sbp,dbp,temp,hct,hb,...,mch,mchc,mcv,ph,hco3,po2,pco2,sao2,fio2,pfratio
count,1.579280e+05,157928.000000,129292.000000,129949.000000,111702.000000,125689.000000,125678.000000,106349.000000,10115.000000,10115.000000,...,10115.000000,10115.000000,10115.000000,5852.000000,5845.000000,5862.000000,5863.000000,5837.000000,5634.000000,5635.000000
mean,3.013101e+08,135.648156,18.844422,82.221433,97.713313,125.735904,73.938247,36.951187,31.750588,10.561354,...,31.440415,33.255136,94.505467,7.394155,21.640975,124.295172,35.398414,96.772349,33.115016,447.233274
std,5.534221e+05,368.719825,4.756056,17.262452,2.806460,19.469239,14.241973,0.499468,6.910072,2.326444,...,2.383141,0.949212,6.149531,0.078771,5.238685,64.937956,10.796019,4.315254,17.450682,291.608062
min,3.000010e+08,-1069.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.700000,5.400000,1.500000,...,19.300000,27.200000,67.300000,6.904000,4.100000,19.800000,6.800000,28.700000,20.000000,34.800000
25%,3.008754e+08,21.000000,16.000000,70.000000,96.000000,112.000000,65.000000,36.600000,26.500000,8.800000,...,30.100000,32.700000,90.900000,7.350000,18.800000,78.900000,28.800000,95.800000,20.900000,258.700000
50%,3.015201e+08,52.000000,18.000000,80.000000,98.000000,125.000000,73.000000,36.900000,31.200000,10.300000,...,31.400000,33.300000,94.100000,7.404000,21.600000,101.300000,33.700000,97.900000,24.000000,387.700000
75%,3.018104e+08,124.000000,21.000000,92.000000,100.000000,139.000000,82.000000,37.300000,36.800000,12.300000,...,32.700000,33.800000,97.800000,7.448000,24.300000,153.175000,40.100000,99.300000,40.000000,534.250000
max,3.018377e+08,4289.000000,250.000000,479.000000,100.000000,369.000000,850.000000,42.000000,63.600000,19.300000,...,47.600000,37.800000,150.200000,7.845000,52.700000,528.400000,200.000000,100.000000,100.000000,2528.100000
